Alot of this code comes from but is revised from
- nanogpt-lecture: [Andrej Karpathy, Github ng-video-lecture]( https://github.com/karpathy/ng-video-lecture?tab=readme-ov-file )

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
torch.__version__

'2.2.1'

In [3]:
import pandas as pd
import data_process as dp

dp.pdb_to_3Di_csv("../data", "./data.csv")
df = pd.read_csv("./data.csv")

In [4]:
train, val = dp.get_train_val_split(df["3Di"].tolist(), ratio=0.9)

In [121]:

batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
vocab_size = 20
max_iters = 200
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dp.get_batch(train if split == "train" else val, batch_size=batch_size, block_size=block_size, device=device)
            logits, loss, _ = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


class CausalSelfAttention(nn.Module):
    def __init__(self, n_embd, n_head, bias, dropout):
        super().__init__()
        assert n_embd % n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class Block(nn.Module):
    def __init__(self, n_embd, n_head, bias, dropout):
        super().__init__()
        self.ln_1 = nn.LayerNorm(n_embd, bias=bias)
        self.attn = CausalSelfAttention(n_embd, n_head, bias, dropout)
        self.ln_2 = nn.LayerNorm(n_embd, bias=bias)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd, bias=bias),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd, bias=bias),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):
    def __init__(self, vocab_size, n_embd, n_head, bias=True, n_layer=4, dropout=0.0):
        super().__init__()
        self.token_embedding_table = nn.Embedding(
            vocab_size, n_embd
        )  # (vocab_size, n_embd) (n_embd is just Channels C)
        self.position_embedding_table = nn.Embedding(
            block_size, n_embd
        )  # so each character knows where it is in the sequence
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, bias, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        """idx is the context and is shape (B, T)"""
        B, T = idx.shape

        # here we select out embeddings for all the tokens up to T and encode positions
        tok_embd = self.token_embedding_table(idx)  # (B, T, C)
        pos_embd = self.position_embedding_table(
            torch.arange(T, device=device)
        )  # select vectors based on context positions
        x = tok_embd + pos_embd

        # ATN 
        embedding = self.blocks(x)
        x = self.ln_f(embedding)

        # MLP at end
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            # generally speaking, this only happens during training
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(
                B * T
            )  # each index is the target of each row in logits
            loss = F.cross_entropy(logits, targets)

        # logits is shape (B, T, C) at inference time and (B*T, C) at train
        return logits, loss, embedding
    
    def seq_embeddings(self, seq):
        all_embeds = []
        B, T = seq.shape
        for i in range(0, seq.shape[1], block_size):
            d = seq[:, i:i+block_size]
            _, _, embeddings = self(d)
            embed = embeddings[:, -1, :]  # pluck out the last embeddings (full block_size)
            all_embeds.append(embed)
        return torch.stack(all_embeds).squeeze()

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss, embedding = self(idx_cond)
            print("here", embedding.shape)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [122]:
model = GPT(
	vocab_size=20,
	n_embd = 64,
	n_head = 4,
	n_layer = 4,
	bias=False,
	dropout=0.0
)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

0.203924 M parameters


In [125]:
context = torch.zeros((1, 156), dtype=torch.long, device=device)
m.seq_embeddings(context)

tensor([[ 2.6182e+00, -3.2591e+00, -9.9736e-01,  2.5315e-01,  1.1520e-01,
          3.6852e-01,  1.2804e+00,  1.9150e+00,  4.7864e-01, -1.6801e+00,
          7.4739e-02, -1.3687e+00, -1.3968e+00, -2.1360e+00, -1.0572e+00,
         -7.7320e-01, -1.3419e+00,  1.5988e+00, -1.0121e+00, -3.5180e-02,
          6.3013e-01,  2.7030e-01,  6.5997e-01,  1.8170e-02, -1.9497e+00,
          3.0305e+00,  2.4125e+00, -3.9176e-01,  2.4479e-02,  7.7176e-01,
         -1.2346e+00, -4.1104e-02,  1.0423e+00,  9.4695e-01, -2.6691e-01,
          1.2325e+00, -1.4351e+00,  1.1765e+00,  3.6361e+00,  1.2605e+00,
         -3.5517e-01,  2.4124e+00,  4.3375e-01, -9.5031e-02, -2.1172e+00,
         -2.4284e+00, -1.5201e-01,  1.0578e+00, -2.1429e+00,  4.8645e+00,
          1.9167e+00, -1.0108e+00, -9.8220e-01, -8.1898e-02,  5.7385e-01,
         -6.0142e-03, -1.4611e+00, -1.1618e+00, -5.3394e-01,  2.0674e+00,
          8.4701e-01, -2.0939e+00, -6.7060e-01, -7.4583e-01],
        [ 2.6182e+00, -3.2591e+00, -9.9736e-01,  2

In [31]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = dp.get_batch(train, batch_size=batch_size, block_size=block_size, device=device)

    # evaluate the loss
    logits, loss, _ = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 0.7168, val loss 2.1117
step 100: train loss 0.5621, val loss 2.8161
step 199: train loss 0.4653, val loss 3.4069


In [32]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(dp.decode(m.generate(context, max_new_tokens=2)[0].tolist()))

here torch.Size([1, 1, 64])
here torch.Size([1, 2, 64])
ADP


In [93]:
B = 2
T = 5
N = 3
a = [torch.zeros((B,T)) for i in range(N)]
a

[tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]])]

In [94]:
# want a to be B, N, T
torch.stack(a).reshape(B, -1, T).shape

torch.Size([2, 3, 5])